<a href="https://colab.research.google.com/github/j-chu33/lab-6/blob/main/DATA_301_Lab_6B_Jonathan_Chu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bing Maps API

In this part of the lab, you will join the CityBikes data from the previous part of the lab with additional data that you will query from the Bing Maps API.

First, you will need to register for a Bing Maps Key. Follow the instructions [here](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key). You should be able to sign in with your Cal Poly account. When you get to Step 4, select:

- Key type: Basic
- Application type: Dev/Test

You will be able to make 125000 free requests with the resulting API key. This should be more than enough to complete this assignment.

We will be working with the [REST services in the Bing Maps API](https://docs.microsoft.com/en-us/bingmaps/rest-services/). Click on the link for a complete documentation of the features.

In [ ]:
key = 'Apo9BGIaIU8pfzLvnFsgCghaNHe0bgs6h20pcmTPqRveovg6XbbyPFuiBsTvYwDY'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Question 1

Read in the `DataFrame` of bike stations in the United States from Part A of this lab. Restrict to the stations in the "Bay Wheels" network (with network ID "ford-gobike").

How many of these stations are in the city/county of San Francisco? 

(_Hint:_ Use the [Locations API](https://docs.microsoft.com/en-us/bingmaps/rest-services/locations/) to get the address associated with each latitude and longitude coordinate.)

In [ ]:
# http://dev.virtualearth.net/REST/v1/Locations/47.64054,-122.12934?o=xml&key={Apo9BGIaIU8pfzLvnFsgCghaNHe0bgs6h20pcmTPqRveovg6XbbyPFuiBsTvYwDY}

In [ ]:
import requests
from lxml import etree
import xml.etree.ElementTree as ET
import json

In [ ]:
import pandas as pd
df_stations = pd.read_csv('https://raw.githubusercontent.com/j-chu33/lab-6/main/stations.csv')

In [ ]:
df_baywheels = df_stations[df_stations['stat.id'] == 'bay-wheels']

In [ ]:
url_lst = []

for i in range(df_baywheels.shape[0]):
  lat = df_baywheels.iloc[i]['latitude']
  lng = df_baywheels.iloc[i]['longitude']
  url = "http://dev.virtualearth.net/REST/v1/Locations/" + str(lat) + ',' + str(lng) + "?o=xml&key=Apo9BGIaIU8pfzLvnFsgCghaNHe0bgs6h20pcmTPqRveovg6XbbyPFuiBsTvYwDY"
  url_lst.append(url)

In [ ]:
import time

addy_lst = []
n = 0.001

for url in url_lst:
  response = requests.get(url)

  root = ET.fromstring(response.text)

  ns = {'ns0': 'http://schemas.microsoft.com/search/local/ws/rest/v1'}
  formatted_address = root.find('.//ns0:FormattedAddress', ns).text

  addy = formatted_address.split(',')

  addy_lst.append(addy[1])

  n += .0001
  time.sleep(n)


In [ ]:
df_addy = pd.DataFrame(addy_lst)
df_addy.value_counts()

 San Francisco    293
 San Jose          83
 Oakland           77
 Berkeley          36
 Emeryville        17
dtype: int64

293 of the 506 are actually in San Francisco.

## Question 2

You want to go to Coit Tower. To save money, you decide to ride a "Bay Wheels" bike to the closest station and hail a cab from there to Coit Tower. What station should you bike to so that you are as close to Coit Tower as possible (as measured by driving distance)? Does your answer agree with the one that you obtained in Part A of this lab? If not, why does it differ?

_Hints:_ 
- You should restrict your attention to bike stations that are in San Francisco, which you determined in Question 1. 
- Use the [Routes API](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix) to calculate a distance matrix between Coit Tower and the bike stations. 
- You can do this with just one call to the API. Because there are too many stations, it is impossible to specify all the locations in the URL. Instead, you should make a POST request (`requests.post`), passing in the parameters as a JSON object through the `json=` parameter of `requests.post`. Read the API documentation carefully to learn how to use the POST API.

In [ ]:
# create json file of orgins and destination lats and logs
df_destinations = pd.DataFrame()
dest_lats = df_baywheels['latitude']
dest_longs = df_baywheels['longitude']
url_dict = {'origin':'', 
            'destination':''}

In [ ]:
lat_long_lst = []

for i in range(df_baywheels.shape[0]):
  dicts = {'latitude':df_baywheels['latitude'].iloc[i], 'longitude':df_baywheels['longitude'].iloc[i]}
  lat_long_lst.append(dicts)

In [ ]:
coit_dict = {'latitude':37.8024, 'longitude':-122.4058}

coit_lst = []
coit_lst.append(coit_dict)

In [ ]:
url_dict = {}
url_dict['origins'] = coit_lst
url_dict['destinations'] = lat_long_lst
url_dict['travelMode'] = 'driving'

In [ ]:
url_dict['origins']

[{'latitude': 37.8024, 'longitude': -122.4058}]

In [ ]:
url_dict['origins']

[{'latitude': 37.8024, 'longitude': -122.4058}]

In [ ]:
url = 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrixAsync?key=Apo9BGIaIU8pfzLvnFsgCghaNHe0bgs6h20pcmTPqRveovg6XbbyPFuiBsTvYwDY'

loc_data = requests.post(url, json=url_dict)

print(loc_data.json())

{'authenticationResultCode': 'ValidCredentials', 'brandLogoUri': 'http://dev.virtualearth.net/Branding/logo_powered_by.png', 'copyright': 'Copyright © 2023 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.', 'resourceSets': [{'estimatedTotal': 1, 'resources': [{'__type': 'RouteProxyAsyncResult:http://schemas.microsoft.com/search/local/ws/rest/v1', 'callbackInSeconds': 30, 'callbackUrl': 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrixAsyncCallback?key=Apo9BGIaIU8pfzLvnFsgCghaNHe0bgs6h20pcmTPqRveovg6XbbyPFuiBsTvYwDY&requestId=ac14504b-0748-4d40-8f28-4cd3be4cf9e5&travelMode=Driving', 'isAccepted': True, 'isCompleted': False, 'requestId': 'ac14504b-0748-4d40-8f28-4cd3be4cf9e5'}]}], 'statusCode': 200, 'statusDescription': 'OK', 'traceId': 'da41483b7130494d8106de0d1762e29b|BN0000644D|0.0.0.0|BNZ0017591'

In [ ]:
callback_url = 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrixAsyncCallback?key=Apo9BGIaIU8pfzLvnFsgCghaNHe0bgs6h20pcmTPqRveovg6XbbyPFuiBsTvYwDY&requestId=ac14504b-0748-4d40-8f28-4cd3be4cf9e5&travelMode=Driving'

response = requests.get(callback_url)

In [ ]:
result = json.loads(response.content.decode('utf-8-sig'))

result

{'authenticationResultCode': 'ValidCredentials',
 'brandLogoUri': 'http://dev.virtualearth.net/Branding/logo_powered_by.png',
 'copyright': 'Copyright © 2023 Microsoft and its suppliers. All rights reserved. This API cannot be accessed and the content and any results may not be used, reproduced or transmitted in any manner without express written permission from Microsoft Corporation.',
 'resourceSets': [{'estimatedTotal': 1,
   'resources': [{'__type': 'RouteProxyAsyncResult:http://schemas.microsoft.com/search/local/ws/rest/v1',
     'callbackInSeconds': -1,
     'callbackUrl': 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrixAsyncCallback?key=Apo9BGIaIU8pfzLvnFsgCghaNHe0bgs6h20pcmTPqRveovg6XbbyPFuiBsTvYwDY&requestId=ac14504b-0748-4d40-8f28-4cd3be4cf9e5&travelMode=Driving',
     'isAccepted': True,
     'isCompleted': True,
     'requestId': 'ac14504b-0748-4d40-8f28-4cd3be4cf9e5',
     'resultUrl': 'https://routematrixpremium.blob.core.windows.net/finalresults/ac14504b-0748-

In [ ]:
resultUrl = 'https://routematrixpremium.blob.core.windows.net/finalresults/ac14504b-0748-4d40-8f28-4cd3be4cf9e5'

response = requests.get(resultUrl)

In [79]:
route_data = json.loads(response.content)

In [91]:
df_results = json_normalize(route_data['results'])

df_results['travelDuration'].idxmin()

<ipython-input-91-a5fd8c602857>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  df_results = json_normalize(route_data['results'])


391

In [92]:
df_baywheels.iloc[391]

Unnamed: 0                                                7948
empty_slots                                               10.0
free_bikes                                                 8.0
id_x                          65134e1e5e8d3b200c458d533e7d544c
latitude                                             37.802767
longitude                                          -122.413276
name                                Lombard St at Columbus Ave
timestamp                          2023-02-24T01:38:55.383000Z
extra.address                                              NaN
extra.last_updated                                1677202710.0
extra.renting                                              1.0
extra.returning                                            1.0
extra.uid                 7b2083d3-b1bb-4ef3-ba53-bebc831bd0a4
stat.id                                             bay-wheels
extra.altitude                                             NaN
extra.ebikes                                           

The station that is identified as the closest is Lombard St at Columbus Ave. This station is different from the one found in part a. It differs because of the distance metric being used. In part A, we used Manhattan distance, in this situation, we do not.

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6B - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.